In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import openpyxl as opx
from tqdm import tqdm

In [2]:
org2  = pd.read_csv('Org2_filtr_by_income.csv')
org2.drop_duplicates(subset=['inn'])
df = org2[['inn', 'income', 'short_name', 'full_name', 'region', 'city', 'lat;lon']]
df = df.sort_values('income', ascending=False)
df

,inn,income,short_name,full_name,region,city,lat;lon
290837,4824095713,33821096000000,ООО &quot;АЛЬТЕРНАТИВА&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,48,Г. ЛИПЕЦК,52.600951;39.5658043
290836,4826133749,28276417000000,ООО &quot;ТЕТРАНИУМ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,48,Г. ЛИПЕЦК,52.6008681;39.5917492
290835,4825131001,11279905000000,ООО &quot;ТРЕЙДИНГ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,48,Г. ЛИПЕЦК,52.5379995;39.585263571415474
290834,7736050003,5179549285000,NaN,ГАЗПРОМ ОАО,15,ГОРОД ВЛАДИКАВКАЗ,43.024593;44.68211
290833,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (НБ)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.8211021;60.6494182
...,...,...,...,...,...,...,...
3,5405477070,50002000,ООО &quot;АРСЕНАЛ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,54,ГОРОД НОВОСИБИРСК,55.0115196;82.9378134
5,5610234680,50002000,ООО &quot;АРК-ЦЕНТР&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,56,NaN,51.1985379;58.3070998
2,4717010585,50001000,ООО &quot;ГЕФЕС&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,47,Г. СЛАНЦЫ,59.1247154;28.0860501
1,3664006076,50001000,ГАУ &quot;УЧЕБНЫЙ АВТОКОМБИНАТ&quot;,ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ ВОРОНЕЖС...,36,Г. Павловск,50.4574176;40.1051697


In [3]:
downloaded = glob('data/*')
inns = list(map(lambda line : int(line[5:]), downloaded))

valid = list()
err = list()
for inn in tqdm(inns):
    path = f'data/{inn}/{inn}_report.xlsx'
    if os.path.isfile(path):
        ws = opx.load_workbook(path).active
        years = [ws.cell(row=8, column=ind).value for ind in range(3,20)]
        years = list(filter(None, years))
        # print(inn)
        
        maxYear = 0
        try:
            maxYear = int(years[-1])
            if int(years[-1]) >= 2021:
                valid.append(inn)
        except ValueError:
            if '2021' or '2022' in years[-1]:
                valid.append(inn)

100%|██████████| 7369/7369 [01:00<00:00, 120.87it/s]


In [4]:
df = df.loc[df['inn'].isin(valid)]

In [5]:
ll = df['lat;lon'].values
lat = []
lon = []
for el in tqdm(ll):
    temp = el.split(';')
    lat.append(float(temp[0]))
    lon.append(float(temp[1]))
    
df['latitude'] = lat
df['longitude'] = lon
df = df.drop(columns=['lat;lon'])
df

100%|██████████| 5013/5013 [00:00<00:00, 2085296.63it/s]
/var/folders/r_/xvc889350jb1wjh3s9_zfq4m0000gn/T/ipykernel_54474/1460109660.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = lat
/var/folders/r_/xvc889350jb1wjh3s9_zfq4m0000gn/T/ipykernel_54474/1460109660.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = lon


,inn,income,short_name,full_name,region,city,latitude,longitude
290834,7736050003,5179549285000,NaN,ГАЗПРОМ ОАО,15,ГОРОД ВЛАДИКАВКАЗ,43.024593,44.682110
290833,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (НБ)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.821102,60.649418
290832,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (АЗС)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.821102,60.649418
290831,5504036333,2070008652000,ООО &quot;МАЛО-ЯМАЛЬСКОЕ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,89,ГОРОД НОВЫЙ УРЕНГОЙ,66.092257,76.686521
290829,8602060555,1524947700000,ПАО &quot;СУРГУТНЕФТЕГАЗ&quot;,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО &quot;СУРГУТНЕФ...,86,Г. СУРГУТ,61.250704,73.367743
...,...,...,...,...,...,...,...,...
283276,6829045090,2413008000,ООО &quot;РАЙ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,68,Г. ТАМБОВ,52.724776,41.453203
283273,4027131618,2411734000,ООО &quot;МАНУФАКТУРЫ БОСКО&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,40,Г. КАЛУГА,54.536984,36.284765
283272,6452109727,2411636000,ООО &quot;НИТА-ФАРМ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,64,Г. САРАТОВ,52.005230,47.810685
283271,5204012980,2411567000,ООО &quot;ННПП-2&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,52,NaN,55.783750,44.776319


In [6]:
df.to_csv('company_df.csv')

In [21]:
df = pd.read_csv('company_df.csv')
df.reset_index(drop=True)
inn_list = df['inn']


debt = []
params_list = [[] for i in range(9)]
err = []
for inn in tqdm(inn_list):
    ws = opx.load_workbook(f'data/{inn}/{inn}_report.xlsx').active
    years = [ws.cell(row=8, column=ind).value for ind in range(3,15)]
    years = list(filter(None, years))

    LAST_COL = len(years) - 1
    
    try:
        debt.append(int(ws.cell(row=35, column=LAST_COL).value) + int(ws.cell(row=40, column=LAST_COL).value))
    except (ValueError, TypeError):
        debt.append(None)
        err.append(inn)
        
    for val_list, row_num in  zip(params_list, [10, 14, 24, 47, 49, 55, 56, 59, 61]):
        try:
            val_list.append(int(ws.cell(row=row_num, column=LAST_COL).value))
        except (ValueError, TypeError):
            val_list.append(None)
            
            
print(len(err))

100%|██████████| 5013/5013 [00:46<00:00, 107.00it/s]

18


In [22]:
df['Debt'] = debt

names = ['Non Material', 'Deferred Assets', 'Assets', 'Revenue', 'Gross profit', 'Interest profit', 'Interest expences', 'EBIT', 'Profit']
for i in range(9):
    df[names[i]] = params_list[i]
    
# df = pd.read_csv('company_fin_df.csv')
df['Interest rate'] = df['Interest expences'] / df['Debt']
df['Material Assets'] = df['Assets'] - df['Non Material'] - df['Deferred Assets']

df

,Unnamed: 0,inn,income,short_name,full_name,region,city,latitude,longitude,Debt,...,Deferred Assets,Assets,Revenue,Gross profit,Interest profit,Interest expences,EBIT,Profit,Interest rate,Material Assets
0,290834,7736050003,5179549285000,NaN,ГАЗПРОМ ОАО,15,ГОРОД ВЛАДИКАВКАЗ,43.024593,44.682110,4.958626e+12,...,2.345270e+11,1.598708e+13,1.126350e+12,4.501800e+11,1.279672e+10,2.748859e+10,-3.806943e+11,-3.062305e+11,0.005544,1.573605e+13
1,290833,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (НБ)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.821102,60.649418,1.456166e+12,...,5.350425e+09,1.823952e+12,4.849752e+11,6.266542e+10,1.032000e+10,1.341651e+10,-2.998806e+09,-2.802774e+09,0.009214,1.816013e+12
2,290832,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (АЗС)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.821102,60.649418,1.456166e+12,...,5.350425e+09,1.823952e+12,4.849752e+11,6.266542e+10,1.032000e+10,1.341651e+10,-2.998806e+09,-2.802774e+09,0.009214,1.816013e+12
3,290831,5504036333,2070008652000,ООО &quot;МАЛО-ЯМАЛЬСКОЕ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,89,ГОРОД НОВЫЙ УРЕНГОЙ,66.092257,76.686521,1.456166e+12,...,5.350425e+09,1.823952e+12,4.849752e+11,6.266542e+10,1.032000e+10,1.341651e+10,-2.998806e+09,-2.802774e+09,0.009214,1.816013e+12
4,290829,8602060555,1524947700000,ПАО &quot;СУРГУТНЕФТЕГАЗ&quot;,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО &quot;СУРГУТНЕФ...,86,Г. СУРГУТ,61.250704,73.367743,1.956224e+11,...,2.846012e+09,3.743467e+12,3.198884e+11,9.237023e+10,2.207062e+10,0.000000e+00,7.384360e+10,6.208008e+10,0.000000,3.740044e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5008,283276,6829045090,2413008000,ООО &quot;РАЙ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,68,Г. ТАМБОВ,52.724776,41.453203,8.043890e+08,...,0.000000e+00,8.217650e+08,1.380262e+09,8.739400e+07,0.000000e+00,0.000000e+00,5.010000e+06,4.008000e+06,0.000000,8.217650e+08
5009,283273,4027131618,2411734000,ООО &quot;МАНУФАКТУРЫ БОСКО&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,40,Г. КАЛУГА,54.536984,36.284765,1.627681e+09,...,3.299100e+07,2.206144e+09,9.974500e+07,-1.183800e+07,2.408500e+07,1.690200e+07,-1.061560e+08,-8.642600e+07,0.010384,2.173153e+09
5010,283272,6452109727,2411636000,ООО &quot;НИТА-ФАРМ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,64,Г. САРАТОВ,52.005230,47.810685,2.625680e+08,...,1.706000e+06,1.751181e+09,1.891464e+09,9.647840e+08,2.300000e+04,1.473600e+07,3.649810e+08,2.879410e+08,0.056123,1.745913e+09
5011,283271,5204012980,2411567000,ООО &quot;ННПП-2&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,52,NaN,55.783750,44.776319,4.751308e+09,...,1.303320e+08,5.653266e+09,2.082859e+09,3.975990e+08,4.524000e+06,2.789890e+08,4.218900e+07,6.401100e+07,0.058718,5.522934e+09


In [24]:
df.to_csv('company_fin_df.csv')